In [5]:
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python

In [26]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files

In [27]:
K = np.array([[2264.91561, 0, 1283.24475],
              [0, 2273.53741, 890.85699],
              [0, 0, 1]])

In [56]:
def draw_cube(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)

    # Draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0,255,0), 3)

    # Draw pillars in blue color
    for i,j in zip(range(4), range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255,0,0), 3)

    # Draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0,0,255), 3)

    return img

# Load image
image = cv2.imread("/content/april3.jpeg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
#gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# List of dictionaries to try
dictionaries = [
    cv2.aruco.DICT_4X4_50,
    cv2.aruco.DICT_4X4_100,
    cv2.aruco.DICT_4X4_250,
    cv2.aruco.DICT_5X5_50,
    cv2.aruco.DICT_5X5_100,
    cv2.aruco.DICT_5X5_250,
    cv2.aruco.DICT_6X6_50,
    cv2.aruco.DICT_6X6_100,
    cv2.aruco.DICT_6X6_250,
    cv2.aruco.DICT_7X7_50,
    cv2.aruco.DICT_7X7_100,
    cv2.aruco.DICT_7X7_250
]

for dict_type in dictionaries:
    # Create ArUco dictionary
    aruco_dict = cv2.aruco.getPredefinedDictionary(dict_type)

    # Create ArUco parameters
    parameters = cv2.aruco.DetectorParameters()

    # Adjust parameters
    parameters.adaptiveThreshConstant = 7
    parameters.adaptiveThreshWinSizeMin = 3
    parameters.adaptiveThreshWinSizeMax = 23
    parameters.adaptiveThreshWinSizeStep = 10
    parameters.minMarkerPerimeterRate = 0.03
    parameters.maxMarkerPerimeterRate = 4.0
    parameters.polygonalApproxAccuracyRate = 0.05
    parameters.minCornerDistanceRate = 0.05
    parameters.minDistanceToBorder = 3

    # Create ArUco detector
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

    # Detect markers
    corners, ids, rejected = detector.detectMarkers(gray)

    # If markers are detected, draw them and break the loop
    if ids is not None and len(ids) > 0:
        print(f"Markers detected using dictionary: {dict_type}")
        cv2.aruco.drawDetectedMarkers(image, corners, ids)

        # Estimate pose for each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, K, None)

        for i in range(len(ids)):
            # Project 3D points to image plane
            axis = np.float32([[0,0,0], [0,0.05,0], [0.05,0.05,0], [0.05,0,0],
                               [0,0,-0.05], [0,0.05,-0.05], [0.05,0.05,-0.05], [0.05,0,-0.05]])
            imgpts, jac = cv2.projectPoints(axis, rvecs[i], tvecs[i], K, None)

            # Draw cube
            image = draw_cube(image, corners[i], imgpts)

            # Draw axis
            cv2.drawFrameAxes(image, K, None, rvecs[i], tvecs[i], 0.1)

        # Display the result
        cv2.imwrite('output_image.jpg', image)

        break
else:
    print("No ArUco markers detected with any dictionary")
    cv2_imshow(image)  # Display the preprocessed image for debugging

Markers detected using dictionary: 2
